In [1]:
%load_ext autoreload
%autoreload 2

# Exercise 3

<img src="./images/03.png" width=800>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
from utils import train_network, View, set_seed
import mlflow
from torchinfo import summary
import os

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/utils.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns07_3'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise07_3')

<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/mlruns07_1/143507330168611334', creation_time=1750415411076, experiment_id='143507330168611334', last_update_time=1750415411076, lifecycle_stage='active', name='Exercise07_1', tags={}>

In [3]:
torch.backends.cudnn.deterministic = True
set_seed(42)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Dataset and DataLoader

In [ ]:
class AutoencodDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        # x, y = self.dataset.__getitem__(index)
        x, _ = self.dataset[index]
        return  x, x

In [ ]:
train_data = AutoencodDataset(torchvision.datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = torchvision.datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoencodDataset(test_data_xy)
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=batch_size)

## Model

In [ ]:
W = 28
H = 28
n = 64
C =1
classes = 10

In [ ]:
def add_noise(x, device="cpu"):
    return x + torch.distributions.Normal(0, .5).sample(
        sample_shape=torch.Size(x.shape)
    ).to(device)

In [ ]:
class AdditiveGausianNoise(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        if self.training:
            return add_noise(x, device=device)
        else:
            return x

In [ ]:
def get_layer(in_size, out_size):
    """
    in_size: how many neurons/features are coming into this layer
    out_size: how many neurons/outputs this hidden layer should produce
    """
    return nn.Sequential( #Organize the conceptual "block" of a hidden layer into a Sequential object
        nn.Linear(in_size,  out_size),
        nn.BatchNorm1d(out_size),
        nn.ReLU())

### Fully Connected Autoencoder

In [ ]:
dn_encoder_fc = nn.Sequential(
    nn.Flatten(),
    AdditiveGausianNoise(),
    get_layer(D, D*2),
    get_layer(D*2, D*2),
    get_layer(D*2, D*2),
    nn.Linear(D*2,  D*2),
)

dn_decoder_fc = nn.Sequential(
    get_layer(D*2, D*2),
    get_layer(D*2, D*2),
    get_layer(D*2, D*2),
    nn.Linear(D*2,  D),
    View(-1, 1, 28, 28)
)

dn_encode_decode_fc = nn.Sequential(
    dn_encoder_fc,
    dn_decoder_fc
)

### Convolutional Autoencoder

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=None):
    if padding is None:
        # Calculate padding to maintain spatial dimensions for odd kernel sizes with stride 1
        padding = (kernel_size - 1) // 2 
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

In [ ]:
dn_encoder_cnn = nn.Sequential(
    AdditiveGausianNoise(), # Add noise directly to the input image
    
    # Layer 1: C channels -> 64 channels, maintains 28x28
    conv_block(C, 64, kernel_size=3), 
    
    # Layer 2: 64 channels -> 128 channels, maintains 28x28
    conv_block(64, 128, kernel_size=3),
    
    # Layer 3: 128 channels -> 256 channels, maintains 28x28
    conv_block(128, 256, kernel_size=3),
    
    # Layer 4: 256 channels -> 512 channels, maintains 28x28 (latent representation channels)
    nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),

)

# Decoder
# Input: (batch_size, 512, 28, 28)
dn_decoder_cnn = nn.Sequential(
    # Layer 1: 512 channels -> 256 channels, maintains 28x28
    conv_block(512, 256, kernel_size=3),
    
    # Layer 2: 256 channels -> 128 channels, maintains 28x28
    conv_block(256, 128, kernel_size=3),
    
    # Layer 3: 128 channels -> 64 channels, maintains 28x28
    conv_block(128, 64, kernel_size=3),
    
    # Final Layer: 64 channels -> C channels (output image channels)
    # No BatchNorm or ReLU here, typically for the final output layer.
    nn.Conv2d(64, C, kernel_size=3, stride=1, padding=1),
    # Final activation depends on image pixel range (e.g., Sigmoid for [0,1], Tanh for [-1,1])
    nn.Sigmoid() # Assuming output image pixel values are in [0, 1]
)

# Combined Autoencoder
dn_encod_decode_cnn = nn.Sequential(
    dn_encoder_cnn,
    dn_decoder_cnn,
)

## Training

In [ ]:
loss_func = nn.MSELoss()
epochs = 50
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size,
    }

In [ ]:
models = {
    'Denoising_Fully_Connected': dn_encode_decode_fc,
    'Denoising_Convolutional': dn_encod_decode_cnn
}

In [ ]:
for experiment, model in models.items():
    params['experiment'] = experiment
    optimizer = optim.AdamW(model.parameters())
    params['optimizer'] = optimizer.defaults

    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model, inpt_size=(batch_size, C, 28, 28))))
    with mlflow.start_run(nested=True, run_name=experiment):
        mlflow.log_artifact('model_summary.txt')
        mlflow.log_params(params)

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            # checkpoint_file_save='model.pth',
            
        )

<img src="./images/E3_train_loss.png">

<img src="./images/E3_valid_loss.png">

<img src="./images/E3_valid_loss_time.png">